In [1]:
import ast
from datetime import datetime

import pandas as pd
from PIL import Image

from utils import Deepfaune, crop_normalized_bbox_square, predict_batch, class_names

/media/plk/TOSHIBA EXT/animal_classification_app/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
images_directory = '../mgr/data/badger'  # user input
BATCH_SIZE = 50  # user input?
checkpoint_path = 'model/deepfaune_polish_lr4_checkpoint.pt' # user input in advanced?

In [3]:
model_wrapper = Deepfaune(checkpoint_path)
classifier = model_wrapper.model.base_model
transforms = model_wrapper.transforms

Using model in resolution 476x476
CUDA available


In [ ]:
megadetector...

In [4]:
classifier.to('cuda')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

images = pd.read_csv('megadetector_results.csv', index_col=0)
total=len(images)
images['bbox'] = images["bbox"].apply(
    lambda b: ast.literal_eval(b) if isinstance(b, str) else None)

batch = []
paths = []
results = pd.DataFrame({'image': [], 'detected_animal': [], 'confidence': []})
ith_image = 0

for _, row in images.iterrows():
    ith_image += 1
    print(f'{100*ith_image/total:.1f}%: {ith_image} / {total}')
    image_path = row['image_path']

    # only animals
    category = row['category']
    if category != 1:
        results.loc[len(results)] = [image_path, 'empty', 0]
        continue

    # image
    try:
        image = Image.open(image_path).convert("RGB")
        cropped_image = crop_normalized_bbox_square(image, row['bbox'])
    except Exception as e:
        # print(f'Error in image {image_path}: {e}')
        continue

    paths.append(image_path)
    batch.append(cropped_image)

    # run classifier every N images (e.g. 32)
    if len(batch) == BATCH_SIZE:
        preds = predict_batch(classifier, batch, transforms, class_names)
        # if confidence (prediction[0][1]) is less than 0.1, classify as other
        detections = [
            prediction[0][0] if prediction[0][1] > 0.1 else 'other' for prediction in preds]
        confs = [prediction[0][1] for prediction in preds]

        batch_results = pd.DataFrame(
            {'image': paths, 'detected_animal': detections, 'confidence': confs})
        results = pd.concat([results, batch_results], ignore_index=True)
        # if confidence less than threshold: other
        batch = []
        paths = []

if len(batch) > 0:
    preds = predict_batch(classifier, batch, transforms, class_names)
    detections = [
        prediction[0][0] if prediction[0][1] > 0.1 else 'other' for prediction in preds]
    confs = [prediction[0][1] for prediction in preds]

    batch_results = pd.DataFrame(
        {'image': paths, 'detected_animal': detections, 'confidence': confs})
    results = pd.concat([results, batch_results], ignore_index=True)

FileNotFoundError: [Errno 2] No such file or directory: 'megadetector_results.csv'

In [ ]:
now = datetime.now().strftime('%Y_%m_%d_%H_%M')
results.to_csv(f'results_{now}.csv')